In [54]:
import numpy as np
import sympy

In [55]:
perch_length = np.array(
    [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 
     36.5, 36.0, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
     )
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [56]:
class Rnn:
  """
  순환 신경망의 계산
  """
  # 입력 가중치
  input_w = []

  # 이전 출력에 대한 가중치
  before_w = []

  # 출력 가중치
  result_w = []

  # 편향 값 bais
  bias = []

  def cal_rnn(self, input, output, input_w = [], bias = []):
    """
    순환 신경망 은닉층까지의 계산
    args
      input : 입력 데이터
      output : 원하는 출력 형태
    return 
      순환 신경망의 계산 결과
    """
    # 이전 출력에 대한 입력, 첫 부분의 경우 이 값이 0이다.
    before = np.zeros((output.shape[0], output.shape[1]))

    # 이전 출력에 대한 가중치
    before_w = np.random.rand(output.shape[0], output.shape[0])

    self.before_w.append(before_w)

    # 가중치 w의 생성, 
    if not input_w:
      input_w = np.random.rand(output.shape[0], input.shape[0])
    
    if not bias:
      bias = np.random.rand()

    self.input_w.append(input_w)

    self.bias.append(bias)

    return (before_w @ before) + (input_w @ input) + bias

  def cal_result(self, input, output, w = []):
    """
    순환 신경망의 은닉층 출력에서 최종 결과값으로의 계산
    args
      input : 입력 데이터
      output : 원하는 출력 형태
    return 
      행렬 곱 연산 결과
    """
    # 가중치 생성
    if not w:
      result_w = np.random.rand(output.shape[0], output.shape[0])

    self.result_w.append(result_w)

    return result_w @ input

In [57]:
class Activation:
  """
  데이터의 활성화 함수 연산 결과를 얻기 위함
  추가로 역전파 계산에 필요한 활성화 함수의 미분 함수
  """
  # 활성화 함수
  fx = 0
  # 활성화 함수의 미분 함수
  d_fx = 0

  def sigmoid_fun(self, input):
    """
    시그모이드 함수
    자연 상수의 지수 함수를 사용한 0~1 사이의 값 출력
    Args:
      input : 입력값
    Return:
      0~1 사이의 값
    """
    x = sympy.Symbol('x')
    result = []

    self.fx = 1/(1+np.e**(-x))

    self.d_fx = sympy.diff(self.fx, x)

    for i in range(input.shape[0]):
      result.append(self.fx.subs([(x, input[i][0])]))

    return np.array(result).reshape(-1, 1)

  def d_sigmoid_fun(self, input):
    """
    시그모이드 함수의 미분값, 노드의 출력이 입력된다.
    arg
      input : 입력값
    return
      시그모이드 미분 함수의 계산 값
    """
    result = []

    x = sympy.Symbol('x')

    for i in range(input.shape[0]):
      result.append(self.d_fx.subs([(x,input[i][0])]))

    return np.array(result).reshape(-1, 1)

In [58]:
class Error:
  """
  오차 함수와 오차 함수의 미분 함수를 얻을 수 있다.  
  """
  # 오차 함수
  fx = 0
  
  # 오차 함수의 미분값
  d_fx = 0
  
  def RMSE(self, target, predict):
    """
    일반적인 제곱 오차 함수
    args:
      target : 목표 변수
      predict : 예측 변수
    return:
      제곱 오차 평균 값
    """
    result = (predict - target)**2
    self.result = np.mean(result)

    t = sympy.Symbol('t')
    y = sympy.Symbol('y')

    self.fx = 1/2*((y-t)**2)

    self.d_fx = sympy.diff(self.fx, y)

    return self.result

  def d_RMSE(self, target, predict):
    """
    RMSE 의 미분 함수
    args
      target : 목표 변수
      predict : 예측 변수
    return:
      미분 함수의 계산 값
    """
    result = []

    t = sympy.Symbol('t')
    y = sympy.Symbol('y')

    for i in range(target.shape[0]):
      result.append(self.d_fx.subs([(t, target[i][0]), (y, predict[i][0])]))
    
    return np.array(result).reshape(-1, 1)


In [59]:
input = perch_length[:3].reshape(-1,1)

In [60]:
target = perch_length[:4].reshape(-1,1)

In [61]:
output = np.zeros((4,1))

In [62]:
input.shape

(3, 1)

In [63]:
output.shape

(4, 1)

In [64]:
rnn = Rnn()

In [65]:
rnn_hidden_input = rnn.cal_rnn(input, output)
rnn_hidden_input

array([[16.13509278],
       [23.88676665],
       [14.31629311],
       [14.4601489 ]])

In [66]:
act = Activation()

In [67]:
rnn_hidden_output = act.sigmoid_fun(rnn_hidden_input)
rnn_hidden_output

array([[0.999999901685359],
       [0.999999999957722],
       [0.999999393944166],
       [0.999999475147951]], dtype=object)

In [68]:
rnn_result = rnn.cal_result(rnn_hidden_output, output)
rnn_result

array([[1.18203729542417],
       [1.91053482912450],
       [1.48584480676199],
       [1.43518364970557]], dtype=object)

델타 연산

예측 노드의 델타,

오차 함수의 미분값과 동일하다.(활성화 함수로 항등 함수를 사용했기 때문)

In [69]:
err = Error()

In [70]:
result = err.RMSE(target, rnn_result)
result

147.930666766194

In [71]:
result_delta = err.d_RMSE(target, rnn_result)
result_delta

array([[-7.21796270457583],
       [-11.7894651708755],
       [-13.5141551932380],
       [-14.7648163502944]], dtype=object)

예측 노드의 델타를 사용한 result_w 값 업데이트

In [72]:
rnn_hidden_output

array([[0.999999901685359],
       [0.999999999957722],
       [0.999999393944166],
       [0.999999475147951]], dtype=object)

In [73]:
d_result_w = result_delta * rnn_hidden_output
d_result_w

array([[-7.21796199494442],
       [-11.7894651703771],
       [-13.5141470029054],
       [-14.7648086009503]], dtype=object)

예측 노드의 델타를 이용한 은닉층 노드의 delta 값 계산

In [74]:
result_delta

array([[-7.21796270457583],
       [-11.7894651708755],
       [-13.5141551932380],
       [-14.7648163502944]], dtype=object)

In [75]:
rnn.result_w[0]

array([[0.18802295, 0.55568031, 0.12443641, 0.31389788],
       [0.50525982, 0.68218805, 0.71791151, 0.00517594],
       [0.49789167, 0.02542964, 0.52432298, 0.43820112],
       [0.09432367, 0.40647504, 0.82187416, 0.11251135]])

은닉층의 각 노드들의 변화에 따른 오차 함수의 변화량

In [89]:
d_hidden_c = (result_delta.T @ rnn.result_w[0]).T
d_hidden_c

array([[-15.4351427352711],
       [-18.3987013596630],
       [-28.5825731811051],
       [-9.90985203407411]], dtype=object)

위 값을 통한 은닉층 각 노드들의 델타 계산, 활성화 함수의 미분값과의 계산

In [77]:
d_hidden_c

array([[-14.2246138463362],
       [-21.4679680852872],
       [-17.4493064720950],
       [-18.2410923560772]], dtype=object)

In [78]:
act.d_sigmoid_fun(rnn_hidden_output)

array([[0.196611942174128],
       [0.196611933245323],
       [0.196611988306343],
       [0.196611980928352]], dtype=object)

In [79]:
hidden_delta = d_hidden_c * act.d_sigmoid_fun(rnn_hidden_output)
hidden_delta

array([[-2.79672895500516],
       [-4.22085870809721],
       [-3.43074284004534],
       [-3.58641730242535]], dtype=object)

은닉층의 델타값을 통한 input_w 의 계산

In [80]:
hidden_delta

array([[-2.79672895500516],
       [-4.22085870809721],
       [-3.43074284004534],
       [-3.58641730242535]], dtype=object)

In [83]:
rnn.input_w[0]

array([[0.13418625, 0.03504287, 0.93531496],
       [0.89914156, 0.22793664, 0.84754193],
       [0.54702681, 0.11774743, 0.5073341 ],
       [0.39470372, 0.43863229, 0.30915058]])

In [82]:
d_input_w = (hidden_delta.T @ rnn.input_w[0]).T
d_input_w

array([[-7.46271264574641],
       [-3.03717332355474],
       [-9.04245299602057]], dtype=object)

은닉층의 델타값을 통한 before_w 의 계산

In [84]:
rnn.before_w[0]

array([[0.72761951, 0.53149889, 0.03904396, 0.21613048],
       [0.75939745, 0.40200076, 0.29118307, 0.17488802],
       [0.93049705, 0.1024098 , 0.77678084, 0.97373921],
       [0.52391858, 0.70815429, 0.69635269, 0.45566093]])

In [86]:
d_before_w = (hidden_delta.T @ rnn.before_w[0]).T
d_before_w

array([[-10.3115506148098],
       [-6.07432518793251],
       [-6.50058460493769],
       [-6.31747506408580]], dtype=object)